In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import obspy
from IPython.display import Image, display

from utils.misc import print_verification
from utils.plot_all_seismograms import plot_all_seismograms
from utils.plot_ascii_trace import read_ascii_trace

In [ ]:
# Define root_dir only if it is not already defined. This way root_dir stays the same, even if you change the work directory.
try:
    root_dir
except NameError:
    root_dir = Path(os.curdir).absolute()

# SPECFEM bin directory
SPECFEM2D_BIN = root_dir / "specfem2d" / "bin"
# Simulation directory
run_folder = root_dir / "03_Layered_Model"
# Go into run directory
os.chdir(run_folder)
# More readable paths
run_folder = run_folder.relative_to(Path(".").absolute())
output_folder = run_folder / "OUTPUT_FILES"

In [ ]:
def make_output_dir():
    if output_folder.exists():
        shutil.rmtree(output_folder)
    output_folder.mkdir()

# Running the Simulation

In [ ]:
make_output_dir()
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

### Wavefield Plot

In [ ]:
Image(output_folder / "forward_image000000500.jpg")

### Seismograms

In [ ]:
fig, axes = plot_all_seismograms(output_folder)
fig.tight_layout()

# Changing the velocity

In [ ]:
print_verification()

In [ ]:
! grep -v ^# ./DATA/Par_file | grep nbmodels -A 4

In [ ]:
# Double the velocity
! sed -i "s/3500.d0/7000.d0/g" DATA/Par_file
! sed -i "s/2000.00d0/4000.00d0/g" DATA/Par_file

In [ ]:
! grep -v ^# ./DATA/Par_file | grep nbmodels -A 4

In [ ]:
# Rerun the simulations
make_output_dir()
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

In [ ]:
print_verification()

In [ ]:
# Turn it back
! sed -i "s/7000.d0/3500.d0/g" DATA/Par_file
! sed -i "s/4000.00d0/2000.00d0/g" DATA/Par_file